## Overview
This notebook implements the model selection phase of the example. To explain the patterns of daily customer purchases
at the store we run PCA on the trimmed daily sales representation. There are many choices to explain variation and summarize
data. PCA is used here for illustration. Other choices such as support vector data description or matrix factorization based approaches
could be favored by some analysts or teams. The point here is that it is possible to capture annotations about rationale used
to make model selection decisions. This is done after developing the model and logging the observations to KMDS. The basic recipe in the note book is:
1. Read the data representation post the modelling choice decision to trim the representation to only use store inventory that accounts for daily variation - see the modelling choice notebook for details
2. Run PCA on the representation
3. Analyze the results
4. Log the observations with KMDS

## Read the Data

In [1]:
import pandas as pd
fp = "../../kmds/examples/retail_q1_post_data_rep_prep.parquet"
df = pd.read_parquet(fp)

## Run PCA

In [2]:
from sklearn.decomposition import PCA

In [3]:
retail_model = PCA(n_components=30)
retail_model.fit_transform(df)

array([[-344.71607154,  -53.99965392,  -67.59825167, ..., -286.83917082,
         275.63272669,  120.01201478],
       [-197.12701461,   44.14881847, -120.21426165, ...,  240.08521115,
         118.84261618,  257.38997529],
       [-336.6022491 ,   20.45942489, -100.43413909, ...,  -45.17495021,
         -10.95369987,  -12.96591113],
       ...,
       [ 165.28815652,  152.7740629 , -161.67334865, ...,   45.54741999,
          13.39956253,    6.24905219],
       [-235.02849098,  102.91589786,  -78.24813975, ...,  -13.79105622,
          36.47025428,  -69.7452242 ],
       [ -94.96490655,  109.86322628,  -67.8645792 , ...,   57.03845375,
          20.66329073,   70.04697077]])

In [4]:
 retail_model.explained_variance_ratio_.cumsum()

array([0.12280201, 0.21023078, 0.29474311, 0.35655678, 0.40462036,
       0.44959697, 0.4931996 , 0.52644409, 0.5544069 , 0.58047431,
       0.60502989, 0.62725672, 0.64848515, 0.66944552, 0.6888062 ,
       0.70562963, 0.72216714, 0.73750752, 0.75260008, 0.76591706,
       0.77847226, 0.78978622, 0.80069928, 0.81082284, 0.82072503,
       0.83010502, 0.83874124, 0.84689316, 0.85464725, 0.86208972])

## Log the Observations to KMDS

In [5]:
from tagging.tag_types import *
from owlready2 import *
from utils.load_utils import *
from utils.path_utils import *
KNOWLEDGE_BASE = "../../kmds/examples/example_ml_kb_exp_workflow.xml"


In [6]:
onto2 = load_kb(KNOWLEDGE_BASE)
with onto2:
    insts = Workflow.instances()
the_workflow_instance = insts[0]

In [7]:
the_workflow_instance

example_ml_kb_exp_workflow.xml.retail_customer_modelling

In [8]:
ms_obs_list = []
observation_count = 1

ms1 = ModelSelectionObservation(namespace=onto2)
ms1.finding = "For this iteration, PCA was the only modelling approach to explain the variance in daily\
product sales. This is sufficient to illustrate how a model selection workflow is logged with KMDS."
ms1.finding_sequence = observation_count
ms1.model_selection_observation_type = ModelSelectionTags.MODEL_SELECTION_STATEMENT.value
ms_obs_list.append(ms1)
the_workflow_instance.has_model_selection_observations = ms_obs_list
onto2.save(file=KNOWLEDGE_BASE, format="rdfxml")